In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
data = pd.read_csv('~/Desktop/project_data_new/embedding_768_TCGA_COAD.csv')
data.index = data['PatientID']
# drop last 2 columns
data = data.drop(data.columns[-2:], axis=1)

In [3]:
data_target = pd.read_csv('~/Desktop/project_data_new/target_768_avg_expanded.csv')

In [4]:
data_target.index = data_target['Unnamed: 0']
data_target = data_target.drop(['Unnamed: 0'], axis = 1)
# only keep the columns with category in the name
data_target = data_target.loc[:, data_target.columns.str.contains('category')]

In [5]:
data = data[data.index.isin(data_target.index)]
data.shape

(449, 768)

# b_12

In [6]:
X = data.values
# Extract the target column for category_b_12
Y = data_target['category_stromal_62'].values

In [7]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rfe = RFE(rf, n_features_to_select=100)  # Choose top 100 features
X = rfe.fit_transform(X, Y)


In [9]:
# save the selected features to csv 
np.savetxt("./Top_100_features_REF_stromal_62.csv", X, delimiter=",")

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from scipy.stats import randint


# Extract the target column for category_b_12
Y = data_target['category_stromal_62'].values

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(random_state=42)

# Define the hyperparameter distribution (use distributions for random search)
param_dist = {
    'n_estimators': randint(100, 200),            # Fewer trees may generalize better
    'max_depth': randint(10, 20),                 # Lower depth can prevent overfitting
    'min_samples_split': randint(10, 20),         # Increase split to limit tree growth
    'min_samples_leaf': randint(1, 5),            # Require more samples per leaf
    'max_features': ['sqrt', 'log2']
}


# Perform Randomized Search with cross-validation
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=60, 
                                   cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
random_search.fit(X_train, Y_train)

# Get the best model from Randomized Search
best_rf = random_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters:", random_search.best_params_)

# Cross-validation on the training set with the best model
cv_scores = cross_val_score(best_rf, X_train, Y_train, cv=5, scoring='f1_weighted')
print("Cross-validation F1 scores on training set:", cv_scores)
print("Average cross-validation F1 score on training set:", np.mean(cv_scores))

# Fit the model on the training data for the final evaluation on test and train sets
best_rf.fit(X_train, Y_train)

# Evaluate on the training set
Y_train_pred = best_rf.predict(X_train)
print("Classification Report for stromal_62 (Train Set):")
print(classification_report(Y_train, Y_train_pred))

# Evaluate on the test set
Y_test_pred = best_rf.predict(X_test)
print("Classification Report for stromal_62 (Test Set):")
print(classification_report(Y_test, Y_test_pred))


Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best Hyperparameters: {'max_depth': 15, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 12, 'n_estimators': 119}
Cross-validation F1 scores on training set: [0.57037037 0.54711126 0.53826087 0.44280611 0.52648009]
Average cross-validation F1 score on training set: 0.525005740393141
Classification Report for stromal_62 (Train Set):
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00       121
           0       0.98      1.00      0.99       120
           1       1.00      0.97      0.99       118

    accuracy                           0.99       359
   macro avg       0.99      0.99      0.99       359
weighted avg       0.99      0.99      0.99       359

Classification Report for stromal_62 (Test Set):
              precision    recall  f1-score   support

          -1       0.58      0.62      0.60        29
           0       0.56      0.52      0.54   

In [12]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
X = data.values
rf = RandomForestClassifier()
rfe = RFE(rf, n_features_to_select=150)  # Choose top 150 features
X = rfe.fit_transform(X, Y)


In [13]:
# save X as Top_100_features_REF
np.savetxt("Top_150_features_REF_stromal_62.csv", X, delimiter=",")

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from scipy.stats import randint

# X = data.values

# Extract the target column for category_b_12
Y = data_target['category_stromal_62'].values

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(random_state=42)

# Define the hyperparameter distribution (use distributions for random search)
param_dist = {
    'n_estimators': randint(100, 200),            # Fewer trees may generalize better
    'max_depth': randint(10, 20),                 # Lower depth can prevent overfitting
    'min_samples_split': randint(10, 20),         # Increase split to limit tree growth
    'min_samples_leaf': randint(1, 5),            # Require more samples per leaf
    'max_features': ['sqrt', 'log2']
}


# Perform Randomized Search with cross-validation
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=50, 
                                   cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
random_search.fit(X_train, Y_train)

# Get the best model from Randomized Search
best_rf = random_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters:", random_search.best_params_)

# Cross-validation on the training set with the best model
cv_scores = cross_val_score(best_rf, X_train, Y_train, cv=5, scoring='f1_weighted')
print("Cross-validation F1 scores on training set:", cv_scores)
print("Average cross-validation F1 score on training set:", np.mean(cv_scores))

# Fit the model on the training data for the final evaluation on test and train sets
best_rf.fit(X_train, Y_train)

# Evaluate on the training set
Y_train_pred = best_rf.predict(X_train)
print("Classification Report for category_stromal_62 (Train Set):")
print(classification_report(Y_train, Y_train_pred))

# Evaluate on the test set
Y_test_pred = best_rf.predict(X_test)
print("Classification Report for category_stromal_62 (Test Set):")
print(classification_report(Y_test, Y_test_pred))


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Hyperparameters: {'max_depth': 15, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 172}
Cross-validation F1 scores on training set: [0.56746032 0.55790186 0.50993514 0.47295516 0.56934437]
Average cross-validation F1 score on training set: 0.5355193696052591
Classification Report for category_stromal_62 (Train Set):
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00       121
           0       0.98      1.00      0.99       120
           1       1.00      0.98      0.99       118

    accuracy                           0.99       359
   macro avg       0.99      0.99      0.99       359
weighted avg       0.99      0.99      0.99       359

Classification Report for category_stromal_62 (Test Set):
              precision    recall  f1-score   support

          -1       0.58      0.62      0.60        29
           0       0.64    

In [37]:
X = pd.read_csv('Top_150_features_REF.csv', header=None)


In [16]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
X = data.values
rf = RandomForestClassifier()
rfe = RFE(rf, n_features_to_select=50)  # Choose top 100 features
X = rfe.fit_transform(X, Y)


In [17]:
# save X as Top_100_features_REF
np.savetxt("Top_50_features_REF_stromal_62.csv", X, delimiter=",")

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from scipy.stats import randint


# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(random_state=42)

# Define the hyperparameter distribution (use distributions for random search)
param_dist = {
    'n_estimators': randint(100, 150),            # Fewer trees may generalize better
    'max_depth': randint(8, 15),                 # Lower depth can prevent overfitting
    'min_samples_split': randint(20, 25),         # Increase split to limit tree growth
    'min_samples_leaf': randint(10, 15),            # Require more samples per leaf
    'max_features': ['sqrt', 'log2']
}


# Perform Randomized Search with cross-validation
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=80, 
                                   cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
random_search.fit(X_train, Y_train)

# Get the best model from Randomized Search
best_rf = random_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters:", random_search.best_params_)

# Cross-validation on the training set with the best model
cv_scores = cross_val_score(best_rf, X_train, Y_train, cv=5, scoring='f1_weighted')
print("Cross-validation F1 scores on training set:", cv_scores)
print("Average cross-validation F1 score on training set:", np.mean(cv_scores))

# Fit the model on the training data for the final evaluation on test and train sets
best_rf.fit(X_train, Y_train)

# Evaluate on the training set
Y_train_pred = best_rf.predict(X_train)
print("Classification Report with 50 features (Train Set):")
print(classification_report(Y_train, Y_train_pred))

# Evaluate on the test set
Y_test_pred = best_rf.predict(X_test)
print("Classification Report 50 features (Test Set):")
print(classification_report(Y_test, Y_test_pred))


Fitting 5 folds for each of 80 candidates, totalling 400 fits
Best Hyperparameters: {'max_depth': 11, 'max_features': 'log2', 'min_samples_leaf': 13, 'min_samples_split': 23, 'n_estimators': 107}
Cross-validation F1 scores on training set: [0.582638   0.57049383 0.52624113 0.52909878 0.54598368]
Average cross-validation F1 score on training set: 0.5508910844891146
Classification Report with 50 features (Train Set):
              precision    recall  f1-score   support

          -1       0.78      0.74      0.76       121
           0       0.67      0.66      0.66       120
           1       0.70      0.75      0.72       118

    accuracy                           0.72       359
   macro avg       0.72      0.72      0.72       359
weighted avg       0.72      0.72      0.72       359

Classification Report 50 features (Test Set):
              precision    recall  f1-score   support

          -1       0.57      0.59      0.58        29
           0       0.63      0.59      0.61  

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd

# Assuming X and Y_category_b_12 are already defined
# X = data.values
# Y_category_b_12 = data_target['category_b_12'].values

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(random_state=42)

# Define the hyperparameter grid (specific values for each hyperparameter)
param_grid = {
    'n_estimators': list(range(120, 141, 2)),        # Every integer between 100 and 150 with step 10
    'max_depth': list(range(8, 16)),                  # Every integer between 8 and 15
    'min_samples_split': list(range(20, 26)),         # Every integer between 20 and 25
    'min_samples_leaf': list(range(10, 16)),          # Every integer between 10 and 15
    'max_features': ['sqrt', 'log2']                  # Options for max features
}

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1)
grid_search.fit(X_train, Y_train)

# Get the best model from Grid Search
best_rf = grid_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Cross-validation on the training set with the best model
cv_scores = cross_val_score(best_rf, X_train, Y_train, cv=5, scoring='f1_weighted')
print("Cross-validation F1 scores on training set:", cv_scores)
print("Average cross-validation F1 score on training set:", np.mean(cv_scores))

# Fit the model on the training data for the final evaluation on test and train sets
best_rf.fit(X_train, Y_train)

# Evaluate on the training set
Y_train_pred = best_rf.predict(X_train)
print("Classification Report with 50 features (Train Set):")
print(classification_report(Y_train, Y_train_pred))

# Evaluate on the test set
Y_test_pred = best_rf.predict(X_test)
print("Classification Report 50 features (Test Set):")
print(classification_report(Y_test, Y_test_pred))


Fitting 5 folds for each of 6336 candidates, totalling 31680 fits
Best Hyperparameters: {'max_depth': 8, 'max_features': 'log2', 'min_samples_leaf': 13, 'min_samples_split': 20, 'n_estimators': 122}
Cross-validation F1 scores on training set: [0.5690247  0.57064813 0.50930472 0.54328309 0.5462198 ]
Average cross-validation F1 score on training set: 0.5476960905156406
Classification Report with 50 features (Train Set):
              precision    recall  f1-score   support

          -1       0.77      0.75      0.76       121
           0       0.66      0.64      0.65       120
           1       0.71      0.75      0.73       118

    accuracy                           0.71       359
   macro avg       0.71      0.71      0.71       359
weighted avg       0.71      0.71      0.71       359

Classification Report 50 features (Test Set):
              precision    recall  f1-score   support

          -1       0.61      0.59      0.60        29
           0       0.64      0.62      0.6